# Is it a Dict or is it a Set? No, it's a ...

Let's suppose I have a pile of change. Something like this.

![Pile of coins](images/coin-pile.jpg)

The lack of organisation makes it hard to see what's in front of us. Some questions which aren't immediately answerable include:

* have I got enough money to watch [Mary Poppins Returns](https://www.imdb.com/title/tt5028340/)?
* ... and buy some popcorn?
* have I got a pound to use as a shopping trolley token?
* have I got the right change for the parking meter?
* am I richer than you?
* if you and I pool our money how much have we got?
* how many coins are there?
* which denomination appears most often?
* ... etc

Coins are only available in a small number of denominations, and there are *lots* of coins. It follows that there's a lot of repetition. We can start to answer our questions by sorting by value, like this.

![Sorting the pile](images/coin-sorting.jpg)

At this point it became clear I'd managed to mix in some Canadian currency, no doubt a result of my son's school trip a couple of years ago. Oh, and some European coins too.

![Exceptions](images/coin-exceptions.jpg)

Finally, we're done.

![Sorted](images/coin-sorted.jpg)

As you can see, the most common denomination -- just -- is the one pence piece, of which there are 90. The second most common is the five pence, of which there are 85.

Let's use a computer to help us organise the data. Suppose the original unsorted pile is represented as a text file `coins.txt`, where each line holds a single coin.

In [1]:
! head coins.txt && echo --- && tail coins.txt && echo --- && wc -l coins.txt

5
1
1
1
1
5
1
5
1
1
--- 
1
2
2
1
5
5
5
1
1
1
--- 
252 coins.txt


The Unix shell gives us tools to organise the pile of coins. `Sort` piped to `uniq` gives us a tally of how many coins there are of each denomination.

In [2]:
! sort coins.txt | uniq -c

     90 1
     13 10
      8 100
     35 2
     18 20
     85 5
      3 50


We can apply `sort` again to order the results by frequency ...

In [31]:
! sort coins.txt | uniq -c | sort -rn

     90 1
     85 5
     35 2
     18 20
     13 10
      8 100
      3 50


... or indeed by denomination.

In [4]:
! sort coins.txt | uniq -c | sort -n -k 2

     90 1
     35 2
     85 5
     13 10
     18 20
      3 50
      8 100


In Python, we don't need to sort and group. Since coin denominations are hashable, we can create a `dict` keyed by denomination with the counts as values.

In [5]:
coin_counts = {}
coins = [int(coin) for coin in open("coins.txt")]

for coin in coins:
    coin_counts[coin] = coin_counts.get(coin, 0) + 1

coin_counts

{5: 85, 1: 90, 2: 35, 10: 13, 50: 3, 20: 18, 100: 8}

Here, `dict.get()` returns a user supplied default -- `0` in this case -- when the key is missing. This pattern is so common there's a special type of dict to do the job. `Collections.defaultdict` accepts a callable on initialisation, which will then be called to set a value for missing keys. So, using `int` as that callable, and remembering `int() == 0` is `True`, we write:

In [6]:
import collections

coin_counts = collections.defaultdict(int)
for coin in coins:
    coin_counts[coin] += 1

coin_counts

defaultdict(int, {5: 85, 1: 90, 2: 35, 10: 13, 50: 3, 20: 18, 100: 8})

By the way, the Unix sort-and-group approach would also work. It's not idiomatic Python, but it's a useful technique to know about. The `itertools` module provides the grouping function.

In [7]:
import itertools

{coin: len(list(group)) for coin, group in itertools.groupby(sorted(coins))}

{1: 90, 2: 35, 5: 85, 10: 13, 20: 18, 50: 3, 100: 8}

![Coin pile](images/coin-pile.jpg)

I said `itertools.groupby` wasn't idiomatic. The modern idiom isn't `defaultdict` either. When we looked in `collections` for help we should have looked harder!

There's [collections.Counter](https://docs.python.org/3/library/collections.html#collections.Counter), which does exactly what we want.

In [8]:
dosh = collections.Counter(coins)
_

{1: 90, 2: 35, 5: 85, 10: 13, 20: 18, 50: 3, 100: 8}

In [9]:
# Have we a pound for the shopping trolley?
100 in dosh

True

In [10]:
# Have we enough money to see the film?
total = sum(v * n for v, n in dosh.items())
assert total == sum(coins)
total >= 975

True

In [11]:
# And a bucket of popcorn too?
total >= 975 + 495

True

In [12]:
# How many coins of how many denominations?
f"I have {sum(dosh.values())} coins of {len(dosh)} denominations"

'I have 252 coins of 7 denominations'

In [13]:
# Ordered by frequency
dosh.most_common()

[(1, 90), (5, 85), (2, 35), (20, 18), (10, 13), (100, 8), (50, 3)]

In [14]:
# Counter subclasses dict
isinstance(dosh, dict), dosh.keys, dosh.values, dosh.items

(True,
 <function Counter.keys>,
 <function Counter.values>,
 <function Counter.items>)

In [15]:
# Note: Counter.update doesn't overwrite like dict.update does - it rolls in the update
dosh.update({1:66, 200:3})
dosh

Counter({5: 85, 1: 156, 2: 35, 10: 13, 50: 3, 20: 18, 100: 8, 200: 3})

In [16]:
# Note: Counter.fromkeys is not implemented ... despite what the docs say!
help(collections.Counter.fromkeys)

Help on method fromkeys in module collections:

fromkeys(iterable, v=None) method of builtins.type instance
    Create a new dictionary with keys from iterable and values set to value.



In [17]:
collections.Counter.fromkeys('abracadabra')

NotImplementedError: Counter.fromkeys() is undefined.  Use Counter(iterable) instead.

In [18]:
collections.Counter('abracadabra')

Counter({'a': 5, 'b': 2, 'r': 2, 'c': 1, 'd': 1})

We've already mentioned `Counter`s *are* `dict`s, where the values are frequencies (aka counts), defaulted to zero. This alone is highly useful and entirely unsurprising. 

Counters are also like `set`s with repeated elements. In C++, they go by the name of [multiset](https://en.cppreference.com/w/cpp/container/multiset). 

Like sets, they support union, intersection and difference -- via overloads of the bitwise operators: `|`, `&`, `-`. Operator `<` etc can be used to check for subset operations.

**Note**: read the small print before use -- in particular for the union operation. If you're combining `Counter`s, addition is probably what you want.

In [25]:
my_fruit   = collections.Counter({"apples": 3, "bananas": 5, "grapes": 25})
your_fruit = collections.Counter({"apples": 4, "bananas": 3, "grapes": 15})
my_fruit | your_fruit # Fruit, unite! 🍏🍌🍇

Counter({'apples': 4, 'bananas': 5, 'grapes': 25})

In [26]:
my_fruit - your_fruit

Counter({'bananas': 2, 'grapes': 10})

Counters also resemble numbers. They can be added and subtracted. Unary `+` and `-` are supported. Note that although negative values are allowed in Counters, you can't put them in there using subtraction. [Read the documentation](https://docs.python.org/3/library/collections.html#collections.Counter) to find out more about what it means for values to be less than or equal to `0`.

## Counters to represent Prime Factorisation

We can represent a natural number as a product of its prime factors. A Counter would be one way to do this.

In [32]:
def primes():
    'Generate the prime numbers in increasing order'
    # http://code.activestate.com/recipes/117119/
    q = 2
    D = {}
    while True:
        if q not in D:
            yield q
            D[q * q] = [q]
        else:
            for p in D[q]:
                D.setdefault(p + q, []).append(p)
            del D[q]
        q += 1

def prime_factorisation(N):
    factored = collections.Counter()
    for p in primes():
        while N % p == 0:
            factored[p] += 1
            N //= p
        if N == 1:
            return factored
        
prime_factorisation(2019)     

Counter({3: 1, 673: 1})

Using this representation, union and intersection correspond to LCM and GCD.

In [33]:
this, prev = prime_factorisation(123456789), prime_factorisation(987654321) 

In [34]:
this | prev

Counter({3: 2, 3607: 1, 3803: 1, 17: 2, 379721: 1})

In [35]:
this & prev

Counter({3: 2})

## Counters at Christmas

Last December [I had a go](https://github.com/wordaligned/advent-of-code-2018/blob/master/solutions.ipynb) at the [Advent of Code](https://adventofcode.com/2018), a series of 25 Christmas themed puzzles made available one-per-day as a countdown to Christmas.

Obviously, I ended up leaning on the built in containers: dicts, lists, tuples, sets.

I also imported `collections`, and, remarkably, used no fewer than 12 `collections.Counter`s. That's every other day, on average -- and I haven't completed all the puzzles yet!

<a href="https://github.com/wordaligned/advent-of-code-2018/blob/master/solutions.ipynb"><img src="images/advent-of-code.png"/></a>
